In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import myslack
import datetime as dt
import pickle
import time
import re
from tqdm import trange

In [2]:
driver = webdriver.Chrome()  
url = 'http://produce48.mnet.com/pc/profile/0'
driver.get(url) 

In [116]:
review_count = int(driver.find_element_by_xpath('//*[@id="comment"]/div/h2/strong').text)
comment_page_count = int(review_count/20)+1
review_count, comment_page_count

(4574, 229)

In [117]:
def open_all_comment():
    print('이전 댓글 모두 불러오기 시작...')    
    for _ in trange(comment_page_count):
        try:
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
            driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/a').click()
            time.sleep(0.3)
        except Exception as e:
            print(e)
            time.sleep(1)
            pass
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    print('이전 댓글 모두 불러오기 성공!')

In [118]:
def make_produce48_df(trainee_name):
    print(trainee_name+'의 댓글 수: '+str(review_count))
    open_all_comment()
    
    print('making dataframe...')
    df = pd.DataFrame(columns=['User_ID','Comment','Date'])
    for i in trange(review_count):
        try:
            df.loc[len(df)] = { 
            'User_ID':driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/dl[{}]/dd/strong'.format(i+1)).text,
            'Comment':driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/dl[{}]/dd/p'.format(i+1)).text,
            'Date':driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/dl[{}]/dd/span'.format(i+1)).text.split(' ')[0],
            }
        except Exception as e:
            print(e)
            pass
    print('making dataframe done!')
    return df

In [119]:
def make_df_name():
    eng_name = \
    re.sub("[^a-zA-Z]"," ",driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[2]/div[1]/dl[2]/dt/p[2]').text).strip()
    result = eng_name.replace(' ','_')
    return result

name = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[2]/div[1]/dl[2]/dt/p[2]/span').text
agency = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[2]/h4/span').text

make_df_name(), name, agency

('PARK_HAE_YOON', '박해윤', 'FNC')

In [120]:
%time PARK_HAE_YOON = make_produce48_df(name)

  0%|          | 0/229 [00:00<?, ?it/s]

박해윤의 댓글 수: 4574
이전 댓글 모두 불러오기 시작...


100%|█████████▉| 228/229 [10:46<00:02,  2.83s/it]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="comment"]/div/div[2]/div/a"}
  (Session info: chrome=68.0.3440.106)
  (Driver info: chromedriver=2.39.562713 (dd642283e958a93ebf6891600db055f1f1b4f3b2),platform=Mac OS X 10.13.6 x86_64)



  0%|          | 0/4574 [00:00<?, ?it/s]

이전 댓글 모두 불러오기 성공!
making dataframe...


100%|██████████| 4574/4574 [1:11:05<00:00,  1.07it/s]

making dataframe done!
CPU times: user 33.9 s, sys: 2.79 s, total: 36.7 s
Wall time: 1h 21min 52s


In [121]:
PARK_HAE_YOON.shape

(4574, 3)

In [127]:
PARK_HAE_YOON.tail()

,User_ID,Comment,Date
4569,국프국,ㅍㅇㅌ,2018-05-21
4570,토미찡,노래 굿,2018-05-21
4571,Summer,So pretty,2018-05-15
4572,호빵,이쁘네~~♡♡♡,2018-05-15
4573,신철우,"그래요,후회없이최선을~.응원할께요.",2018-05-15


In [128]:
pickle.dump(PARK_HAE_YOON, open('./produce48/FNC/PARK_HAE_YOON.pkl','wb'))

In [129]:
PARK_HAE_YOON_df = pickle.load(open('./produce48/FNC/PARK_HAE_YOON.pkl','rb'))

In [130]:
myslack.send_slack('{} finish!'.format(make_df_name()))

<Response [200]>


In [131]:
PARK_HAE_YOON_df.tail()

,User_ID,Comment,Date
4569,국프국,ㅍㅇㅌ,2018-05-21
4570,토미찡,노래 굿,2018-05-21
4571,Summer,So pretty,2018-05-15
4572,호빵,이쁘네~~♡♡♡,2018-05-15
4573,신철우,"그래요,후회없이최선을~.응원할께요.",2018-05-15


In [663]:
driver.close() # 창닫기
driver.quit() # 브라우져 닫기